In [1]:
import bz2
import xml.sax
import re
import time
import json
import psycopg2
import mwparserfromhell
import nltk
from nltk.corpus import stopwords
import math

# PostgreSQL Database setup 

In [39]:
data_path = '/home/bence/skool/2nd_semester/nlp/exercises/2_wikification/enwiki-20061130-pages-articles.xml.bz2'

In [2]:
connection_data = {
    "database": "wikification",
    "user": "postgres",
    "password": "passwd",
    "host": "127.0.0.1",
    "port": "5432"
}

connection = psycopg2.connect(  database=connection_data['database'],
                                user = connection_data['user'],
                                password = connection_data['password'],
                                host = connection_data['host'],
                                port = connection_data['port'])
crsr = connection.cursor()

In [6]:
# crsr.execute('''DROP TABLE IF EXISTS articles''') 
connection.commit()

In [7]:
crsr.execute('''CREATE TABLE articles
                (ID SERIAL PRIMARY KEY,
                TITLE TEXT NOT NULL,
                TEXTCONTENT TEXT NOT NULL);''')
connection.commit()

In [11]:
crsr.execute(
    "prepare insertarticle as "
    "INSERT INTO articles (TITLE, TEXTCONTENT) VALUES ($1,  $2)"
    )
connection.commit()

DuplicatePreparedStatement: prepared statement "insertarticle" already exists


# Reading the Wiki dump using my own parser

In [12]:
processed_counter = 0
added_to_db_counter = 0

with open('../enwiki-20061130-pages-articles.xml', 'rt') as wiki_dump:
    line = ''
    
    while(line := wiki_dump.readline()):
        title = ''
        text = ''
        
        if re.search('<page>', line):
            while not re.search('</page>', line):
                
                # search for title
                if re.search('<title>', line):
                    title = re.sub('<title>', '', line)
                    title = re.sub('</title>', '', title).strip()
                    
                # search for text content
                if re.search('<text .+>', line):
                    while not re.search('</text>', line):
                        text += line
                        line = wiki_dump.readline()
                    text += line
                    text = re.sub('<text .+>', '', text)
                    text = re.sub('</text>', '', text).strip()
                    
                line = wiki_dump.readline()
                
        # check progress
        if processed_counter % 5000 == 0:
            print("processed %d pages, added %d to database" % (processed_counter, added_to_db_counter))
        
        # after every 1000 article, commit to db
        if added_to_db_counter % 1000 == 0:
            connection.commit()
        
        processed_counter += 1
        
        # after filtering, insert article into table
        if len(title) > 0 and len(text) > 1024:
            crsr.execute("execute insertarticle (%s,  %s)", (title, text))
            added_to_db_counter += 1

processed 0 pages, added 0 to database
processed 5000 pages, added 3151 to database
processed 10000 pages, added 6467 to database
processed 15000 pages, added 9751 to database
processed 20000 pages, added 13060 to database
processed 25000 pages, added 16249 to database
processed 30000 pages, added 19857 to database
processed 35000 pages, added 23056 to database
processed 40000 pages, added 26306 to database
processed 45000 pages, added 29102 to database
processed 50000 pages, added 32179 to database
processed 55000 pages, added 34850 to database
processed 60000 pages, added 37448 to database
processed 65000 pages, added 39704 to database
processed 70000 pages, added 42123 to database
processed 75000 pages, added 45502 to database
processed 80000 pages, added 48387 to database
processed 85000 pages, added 53116 to database
processed 90000 pages, added 57809 to database
processed 95000 pages, added 62348 to database
processed 100000 pages, added 67118 to database
processed 105000 pages, 

processed 845000 pages, added 346719 to database
processed 850000 pages, added 348504 to database
processed 855000 pages, added 350276 to database
processed 860000 pages, added 352086 to database
processed 865000 pages, added 353639 to database
processed 870000 pages, added 355100 to database
processed 875000 pages, added 356300 to database
processed 880000 pages, added 357376 to database
processed 885000 pages, added 358728 to database
processed 890000 pages, added 359842 to database
processed 895000 pages, added 360630 to database
processed 900000 pages, added 362117 to database
processed 905000 pages, added 363610 to database
processed 910000 pages, added 365213 to database
processed 915000 pages, added 366967 to database
processed 920000 pages, added 368733 to database
processed 925000 pages, added 370552 to database
processed 930000 pages, added 372119 to database
processed 935000 pages, added 373731 to database
processed 940000 pages, added 375296 to database
processed 945000 pag

processed 1670000 pages, added 604415 to database
processed 1675000 pages, added 605998 to database
processed 1680000 pages, added 607517 to database
processed 1685000 pages, added 608925 to database
processed 1690000 pages, added 609752 to database
processed 1695000 pages, added 610622 to database
processed 1700000 pages, added 612213 to database
processed 1705000 pages, added 613681 to database
processed 1710000 pages, added 614469 to database
processed 1715000 pages, added 615105 to database
processed 1720000 pages, added 615924 to database
processed 1725000 pages, added 617057 to database
processed 1730000 pages, added 617998 to database
processed 1735000 pages, added 619225 to database
processed 1740000 pages, added 620497 to database
processed 1745000 pages, added 621900 to database
processed 1750000 pages, added 623242 to database
processed 1755000 pages, added 624587 to database
processed 1760000 pages, added 626052 to database
processed 1765000 pages, added 627579 to database


processed 2490000 pages, added 831812 to database
processed 2495000 pages, added 833094 to database
processed 2500000 pages, added 834370 to database
processed 2505000 pages, added 835751 to database
processed 2510000 pages, added 837043 to database
processed 2515000 pages, added 838418 to database
processed 2520000 pages, added 839775 to database
processed 2525000 pages, added 841200 to database
processed 2530000 pages, added 842579 to database
processed 2535000 pages, added 843978 to database
processed 2540000 pages, added 845348 to database
processed 2545000 pages, added 846720 to database
processed 2550000 pages, added 847993 to database
processed 2555000 pages, added 849254 to database
processed 2560000 pages, added 850536 to database
processed 2565000 pages, added 851883 to database
processed 2570000 pages, added 853187 to database
processed 2575000 pages, added 854441 to database
processed 2580000 pages, added 855689 to database
processed 2585000 pages, added 856916 to database


processed 3310000 pages, added 1036335 to database
processed 3315000 pages, added 1037569 to database
processed 3320000 pages, added 1038873 to database
processed 3325000 pages, added 1040120 to database
processed 3330000 pages, added 1041300 to database
processed 3335000 pages, added 1042521 to database
processed 3340000 pages, added 1043658 to database
processed 3345000 pages, added 1044982 to database
processed 3350000 pages, added 1046093 to database
processed 3355000 pages, added 1047299 to database
processed 3360000 pages, added 1048514 to database
processed 3365000 pages, added 1049654 to database
processed 3370000 pages, added 1050799 to database
processed 3375000 pages, added 1051982 to database
processed 3380000 pages, added 1053011 to database
processed 3385000 pages, added 1054148 to database
processed 3390000 pages, added 1055323 to database
processed 3395000 pages, added 1056419 to database
processed 3400000 pages, added 1057478 to database
processed 3405000 pages, added 

processed 4115000 pages, added 1218990 to database
processed 4120000 pages, added 1219973 to database
processed 4125000 pages, added 1220934 to database
processed 4130000 pages, added 1221884 to database
processed 4135000 pages, added 1222703 to database
processed 4140000 pages, added 1223705 to database
processed 4145000 pages, added 1224812 to database
processed 4150000 pages, added 1225754 to database
processed 4155000 pages, added 1226800 to database
processed 4160000 pages, added 1229711 to database
processed 4165000 pages, added 1232102 to database
processed 4170000 pages, added 1233181 to database
processed 4175000 pages, added 1234144 to database
processed 4180000 pages, added 1235136 to database
processed 4185000 pages, added 1236064 to database
processed 4190000 pages, added 1237027 to database


In [13]:
connection.commit()

<hr>

### Processing the rows of the 'articles' table => building word occurence dictionary

In [3]:
word_occurences_dict = {}

In [4]:
def wikilink_cleaner(link):
    link = re.sub('Category:', '', link)
    link = re.sub('Help:', '', link)
    link = re.sub('Extension:', '', link)
    link = re.sub('User:', '', link)
    link = re.sub('Manual:', '', link)
#     link = re.sub('|', '', link)
    link = re.sub('File:', '', link)
    link = re.sub('Special:', '', link)
    link = re.sub('((I|i)mage|IMAGE):.*', '', link)
    link = re.sub('[a-z][a-z]:.*', '', link)
    # ... more cleaning
    
    return link

def get_clean_str_wikilinks(text):
    wiki = mwparserfromhell.parse(text)
    
    wikilinks = [x.title for x in wiki.filter_wikilinks()]
    str_wikilinks = [x.strip_code().strip() for x in wikilinks]
    str_wikilinks = [wikilink_cleaner(x) for x in str_wikilinks]
    
    return str_wikilinks

def put_links_from_article(text):
    clean_links = get_clean_str_wikilinks(text)
    
    for link in clean_links:
        if link in word_occurences_dict:
            word_occurences_dict[link] += 1
        else:
            word_occurences_dict[link] = 1

In [5]:
def process_articles_from_table(limit = None, print_progress_step = 10000):
    start_time = time.time()
    
    # get the number of rows in the 'articles' table
    query = 'SELECT count(id) FROM articles;'
    crsr.execute(query)
    row = crsr.fetchone()
    row_count = int(row[0])

    # get the biggest ID in the 'articles' table
    query = 'SELECT max(id) FROM articles;'
    crsr.execute(query)
    row = crsr.fetchone()
    max_id = int(row[0])

    for i in range(max_id):

        # get a single row from the database
        query = 'SELECT * FROM articles WHERE id= %s;'
        row_id = (str(i), )
        crsr.execute(query, row_id)
        row = crsr.fetchone()

        # error handling
        if row is None:
            continue

        # build the dictionary, format: {'word': # of occurence}
        put_links_from_article(row[2])
        
        if i % print_progress_step == 0:
            print('Processed %d/%d rows, progress: %.3f%%' % (i, row_count, (i/row_count) * 100))

        if i is not None and i == limit:
            print('Early stop at i = %d' % i)
            break

    finish_time = time.time()
    print("\nExecuted in %s seconds" % (finish_time - start_time))

In [9]:
word_occurences_dict = {}
process_articles_from_table(limit = None, print_progress_step = 25000)

# write result to json file
with open('word_occurences.json', 'w') as out_file:
    json.dump(word_occurences_dict, out_file)

Processed 25000/1237973 rows, progress: 2.019%
Processed 50000/1237973 rows, progress: 4.039%
Processed 75000/1237973 rows, progress: 6.058%
Processed 100000/1237973 rows, progress: 8.078%
Processed 125000/1237973 rows, progress: 10.097%
Processed 150000/1237973 rows, progress: 12.117%
Processed 175000/1237973 rows, progress: 14.136%
Processed 200000/1237973 rows, progress: 16.155%
Processed 225000/1237973 rows, progress: 18.175%
Processed 250000/1237973 rows, progress: 20.194%
Processed 275000/1237973 rows, progress: 22.214%
Processed 300000/1237973 rows, progress: 24.233%
Processed 325000/1237973 rows, progress: 26.253%
Processed 350000/1237973 rows, progress: 28.272%
Processed 375000/1237973 rows, progress: 30.291%
Processed 400000/1237973 rows, progress: 32.311%
Processed 425000/1237973 rows, progress: 34.330%
Processed 450000/1237973 rows, progress: 36.350%
Processed 475000/1237973 rows, progress: 38.369%
Processed 500000/1237973 rows, progress: 40.389%
Processed 525000/1237973 ro

In [3]:
with open('word_occurences.json', 'rt') as f:
    word_occurences_dict = json.load(f)

In [4]:
word_occurences_dict

{'political philosophy': 244,
 'government': 2509,
 'state': 1529,
 '29 August': 1297,
 '2006': 596385,
 'etymology': 725,
 'Greek language': 8678,
 'Wiktiona': 6782,
 'archon': 78,
 'rulership': 1,
 'Taoist': 260,
 'Laozi': 94,
 'Zeno of Citium': 56,
 'Stoicism': 153,
 'Enlightenment': 97,
 'Rousseau': 119,
 'abuse': 205,
 'French Revolution': 2987,
 'Jacobin (politics)': 90,
 'William Godwin': 110,
 '': 2890948,
 'Peter Kropotkin': 170,
 'Mary Wollstonecraft': 119,
 'Mary Shelley': 297,
 'Frankenstein': 544,
 'Gustave Courbet': 102,
 'Pierre-Joseph Proudhon': 126,
 'What Is Property?': 1,
 'spontaneous order': 12,
 'Mutualism (economic theory)': 38,
 'What is Property?': 22,
 'labor theory of value': 79,
 'William B. Greene': 20,
 'Josiah Warren': 37,
 'Benjamin R. Tucker': 3,
 'Pierre Leroux': 17,
 'Collectivist anarchism': 6,
 'Mikhail Bakunin': 157,
 "International Workingmen's Association": 52,
 'anarcho-communist': 14,
 'Anarchism in Spain': 72,
 'collectivist anarchism': 13,
 '

# TF-IDF

In [5]:
# read test text
f = open('Loop_trolley.txt', 'r')
text = f.read()
text = text.replace('\n', '')

print(text)

# text cleanup
# 1. tokenization
tokenizer = nltk.RegexpTokenizer(r'\w+')
tokenized_text = tokenizer.tokenize(text)
tokenized_text = [w.lower() for w in tokenized_text]

# 2. filter stopwords
stop_words = set(stopwords.words('english'))
filtered_text = [w for w in tokenized_text if not w in stop_words]

# every keyword from the whole corpus
keywords = set([w.lower() for w in set(word_occurences_dict.keys())])

del word_occurences_dict

# calculate term-frequency
tf_word = {}
for word in filtered_text:
    if word in tf_word:
        tf_word[word] += 1
    else:
        tf_word[word] = 1
        
# corpus size
N = 1237973


n_word = tf_word
for key, val in n_word.items():
    n_word[key] = 0

The Loop Trolley was a 2.2-mile (3.5 km), 10-station heritage streetcar line in St. Louis, Missouri. It operated from November 2018 to December 2019. As of December 2019, the Bi-State Development Agency, a regional transit operator, was seeking ways to restart operations.Opened in November 2018, the Loop Trolley connected the Delmar Loop district and the Missouri History Museum in Forest Park, serving parts of University City and the St. Louis neighborhoods of DeBaliviere Place, Skinker/DeBaliviere, and the West End. It also served two MetroLink light-rail stations: Delmar Loop and Forest Park–DeBaliviere.At its closing, the line operated from noon until 6 p.m. on Thursdays through Sundays. Passengers were carried on two replica-historic streetcars.The Loop Trolley Transportation Development District owns the line and trolley cars, which were operated by the non-profit Loop Trolley Company. Operating funds came from a one-cent sales tax collected by businesses along and near the line, 

In [6]:
def calc_n_word(limit = None, print_progress_step = 10000):
    '''calculate the number of docs where a given word from the text is present'''
    
    start_time = time.time()
    
    # get the number of rows in the 'articles' table
    query = 'SELECT count(id) FROM articles;'
    crsr.execute(query)
    row = crsr.fetchone()
    row_count = int(row[0])

    # get the biggest ID in the 'articles' table
    query = 'SELECT max(id) FROM articles;'
    crsr.execute(query)
    row = crsr.fetchone()
    max_id = int(row[0])

    for i in range(max_id):

        # get a single row from the database
        query = 'SELECT textcontent FROM articles WHERE id= %s;'
        row_id = (str(i), )
        crsr.execute(query, row_id)
        row = crsr.fetchone()

        # error handling
        if row is None:
            continue

        # build the dictionary, format: {'word': # of occurence}
        for key, val in n_word.items():
            if key in row[0].lower():
                n_word[key] += 1
                break
        
        if i % print_progress_step == 0:
            print('Processed %d/%d rows, progress: %.3f%%' % (i, row_count, (i/row_count) * 100))

        if i is not None and i == limit:
            print('Early stop at i = %d' % i)
            break

    finish_time = time.time()
    print("\nExecuted in %s seconds" % (finish_time - start_time))

In [7]:
calc_n_word(print_progress_step = 10000)

Processed 10000/1237973 rows, progress: 0.808%
Processed 20000/1237973 rows, progress: 1.616%
Processed 30000/1237973 rows, progress: 2.423%
Processed 40000/1237973 rows, progress: 3.231%
Processed 50000/1237973 rows, progress: 4.039%
Processed 60000/1237973 rows, progress: 4.847%
Processed 70000/1237973 rows, progress: 5.654%
Processed 80000/1237973 rows, progress: 6.462%
Processed 90000/1237973 rows, progress: 7.270%
Processed 100000/1237973 rows, progress: 8.078%
Processed 110000/1237973 rows, progress: 8.885%
Processed 120000/1237973 rows, progress: 9.693%
Processed 130000/1237973 rows, progress: 10.501%
Processed 140000/1237973 rows, progress: 11.309%
Processed 150000/1237973 rows, progress: 12.117%
Processed 160000/1237973 rows, progress: 12.924%
Processed 170000/1237973 rows, progress: 13.732%
Processed 180000/1237973 rows, progress: 14.540%
Processed 190000/1237973 rows, progress: 15.348%
Processed 200000/1237973 rows, progress: 16.155%
Processed 210000/1237973 rows, progress: 

In [13]:
# calculate tf-idf for every word in n_word
tfidf_dict = n_word
for key, val in tfidf_dict.items():
    
    if n_word[key] == 0:
        continue
        
    tfidf_dict[key] = tf_word[key] * math.log(N/n_word[key])

In [15]:
tfidf_dict = dict(sorted(tfidf_dict.items(), key=lambda item: item[1], reverse=True))
tfidf_dict

{'st': 280601.4491159988,
 '3': 275176.56503862183,
 '2': 265009.9475058322,
 'loop': 197895.76392299408,
 '5': 194620.5921496898,
 'line': 137654.38871794264,
 'mile': 73682.35366935281,
 '10': 70987.0917990709,
 'station': 54457.46265362127,
 'trolley': 53210.14635113441,
 'km': 39060.206790365766,
 'heritage': 19244.688221205965,
 'bi': 8127.0831465699375,
 'p': 4730.073710738936,
 'two': 1591.2872621854194,
 'end': 1428.3060115742624,
 'also': 1428.3060115742624,
 'place': 1260.323457920923,
 'streetcar': 905.1876641140125,
 'city': 905.1876641140125,
 '6': 614.7847857513578,
 'november': 402.29474504002724,
 'missouri': 286.5974081857275,
 'regional': 286.5974081857275,
 'served': 286.5974081857275,
 'rail': 286.5974081857275,
 'december': 159.76013198238994,
 'development': 159.76013198238994,
 'agency': 159.76013198238994,
 'transit': 159.76013198238994,
 'park': 159.76013198238994,
 'serving': 159.76013198238994,
 'parts': 159.76013198238994,
 'university': 159.76013198238994,


In [16]:
top_percent = 15
top_keywords = (len(tfidf_dict) * top_percent) / 100
counter = 0
limit = int(top_keywords)

print("The keywords in the text:")
for key, value in tfidf_dict.items():
    print(key, '\t score:', value)
    
    if counter >= limit:
        break
        
    counter += 1

The keywords in the text:
st 	 score: 280601.4491159988
3 	 score: 275176.56503862183
2 	 score: 265009.9475058322
loop 	 score: 197895.76392299408
5 	 score: 194620.5921496898
line 	 score: 137654.38871794264
mile 	 score: 73682.35366935281
10 	 score: 70987.0917990709
station 	 score: 54457.46265362127
trolley 	 score: 53210.14635113441
km 	 score: 39060.206790365766
heritage 	 score: 19244.688221205965
bi 	 score: 8127.0831465699375
p 	 score: 4730.073710738936


# Keyphraseness

In [17]:
filtered_text

['loop',
 'trolley',
 '2',
 '2',
 'mile',
 '3',
 '5',
 'km',
 '10',
 'station',
 'heritage',
 'streetcar',
 'line',
 'st',
 'louis',
 'missouri',
 'operated',
 'november',
 '2018',
 'december',
 '2019',
 'december',
 '2019',
 'bi',
 'state',
 'development',
 'agency',
 'regional',
 'transit',
 'operator',
 'seeking',
 'ways',
 'restart',
 'operations',
 'opened',
 'november',
 '2018',
 'loop',
 'trolley',
 'connected',
 'delmar',
 'loop',
 'district',
 'missouri',
 'history',
 'museum',
 'forest',
 'park',
 'serving',
 'parts',
 'university',
 'city',
 'st',
 'louis',
 'neighborhoods',
 'debaliviere',
 'place',
 'skinker',
 'debaliviere',
 'west',
 'end',
 'also',
 'served',
 'two',
 'metrolink',
 'light',
 'rail',
 'stations',
 'delmar',
 'loop',
 'forest',
 'park',
 'debaliviere',
 'closing',
 'line',
 'operated',
 'noon',
 '6',
 'p',
 'thursdays',
 'sundays',
 'passengers',
 'carried',
 'two',
 'replica',
 'historic',
 'streetcars',
 'loop',
 'trolley',
 'transportation',
 'developm

In [43]:
for key, val in n_word.items():
    n_word[key] = 0
calc_n_word(print_progress_step = 100000)

Processed 10000/1237973 rows, progress: 0.808%
Processed 20000/1237973 rows, progress: 1.616%
Processed 30000/1237973 rows, progress: 2.423%
Processed 40000/1237973 rows, progress: 3.231%
Processed 50000/1237973 rows, progress: 4.039%
Processed 60000/1237973 rows, progress: 4.847%
Processed 70000/1237973 rows, progress: 5.654%
Processed 80000/1237973 rows, progress: 6.462%
Processed 90000/1237973 rows, progress: 7.270%
Processed 100000/1237973 rows, progress: 8.078%
Processed 110000/1237973 rows, progress: 8.885%
Processed 120000/1237973 rows, progress: 9.693%
Processed 130000/1237973 rows, progress: 10.501%
Processed 140000/1237973 rows, progress: 11.309%
Processed 150000/1237973 rows, progress: 12.117%
Processed 160000/1237973 rows, progress: 12.924%
Processed 170000/1237973 rows, progress: 13.732%
Processed 180000/1237973 rows, progress: 14.540%
Processed 190000/1237973 rows, progress: 15.348%
Processed 200000/1237973 rows, progress: 16.155%
Processed 210000/1237973 rows, progress: 

In [45]:
n_word

{'loop': 15621,
 'trolley': 1724,
 '2': 1123392,
 'mile': 2836,
 '3': 31779,
 '5': 15115,
 'km': 1093,
 '10': 2676,
 'station': 1785,
 'heritage': 403,
 'streetcar': 8,
 'line': 7954,
 'st': 33305,
 'louis': 0,
 'missouri': 2,
 'operated': 0,
 'november': 3,
 '2018': 0,
 'december': 1,
 '2019': 0,
 'bi': 127,
 'state': 0,
 'development': 1,
 'agency': 1,
 'regional': 2,
 'transit': 1,
 'operator': 0,
 'seeking': 0,
 'ways': 0,
 'restart': 0,
 'operations': 0,
 'opened': 0,
 'connected': 0,
 'delmar': 0,
 'district': 0,
 'history': 0,
 'museum': 0,
 'forest': 0,
 'park': 1,
 'serving': 1,
 'parts': 1,
 'university': 1,
 'city': 8,
 'neighborhoods': 0,
 'debaliviere': 0,
 'place': 12,
 'skinker': 0,
 'west': 0,
 'end': 14,
 'also': 14,
 'served': 2,
 'two': 16,
 'metrolink': 0,
 'light': 1,
 'rail': 2,
 'stations': 0,
 'closing': 0,
 'noon': 0,
 '6': 5,
 'p': 63,
 'thursdays': 0,
 'sundays': 0,
 'passengers': 0,
 'carried': 0,
 'replica': 0,
 'historic': 0,
 'streetcars': 0,
 'transporta

In [30]:
n_keyword = n_word
for key, val in n_keyword.items():
    n_keyword[key] = 0

In [31]:
n_keyword

{'loop': 0,
 'trolley': 0,
 '2': 0,
 'mile': 0,
 '3': 0,
 '5': 0,
 'km': 0,
 '10': 0,
 'station': 0,
 'heritage': 0,
 'streetcar': 0,
 'line': 0,
 'st': 0,
 'louis': 0,
 'missouri': 0,
 'operated': 0,
 'november': 0,
 '2018': 0,
 'december': 0,
 '2019': 0,
 'bi': 0,
 'state': 0,
 'development': 0,
 'agency': 0,
 'regional': 0,
 'transit': 0,
 'operator': 0,
 'seeking': 0,
 'ways': 0,
 'restart': 0,
 'operations': 0,
 'opened': 0,
 'connected': 0,
 'delmar': 0,
 'district': 0,
 'history': 0,
 'museum': 0,
 'forest': 0,
 'park': 0,
 'serving': 0,
 'parts': 0,
 'university': 0,
 'city': 0,
 'neighborhoods': 0,
 'debaliviere': 0,
 'place': 0,
 'skinker': 0,
 'west': 0,
 'end': 0,
 'also': 0,
 'served': 0,
 'two': 0,
 'metrolink': 0,
 'light': 0,
 'rail': 0,
 'stations': 0,
 'closing': 0,
 'noon': 0,
 '6': 0,
 'p': 0,
 'thursdays': 0,
 'sundays': 0,
 'passengers': 0,
 'carried': 0,
 'replica': 0,
 'historic': 0,
 'streetcars': 0,
 'transportation': 0,
 'owns': 0,
 'cars': 0,
 'non': 0,
 'pr

In [34]:
def calc_n_keyword(limit = None, print_progress_step = 100000):
    '''calculate the number of docs where a given word from the text is present as a keyword'''
    
    start_time = time.time()
    
    # get the number of rows in the 'articles' table
    query = 'SELECT count(id) FROM articles;'
    crsr.execute(query)
    row = crsr.fetchone()
    row_count = int(row[0])

    # get the biggest ID in the 'articles' table
    query = 'SELECT max(id) FROM articles;'
    crsr.execute(query)
    row = crsr.fetchone()
    max_id = int(row[0])

    for i in range(max_id):

        # get a single row from the database
        query = 'SELECT textcontent FROM articles WHERE id= %s;'
        row_id = (str(i), )
        crsr.execute(query, row_id)
        row = crsr.fetchone()

        # error handling
        if row is None:
            continue

        # build the dictionary, format: {'word': # of occurence}
        for key, val in n_word.items():
            if key in row[0].lower() and key in keywords:
                n_keyword[key] += 1
                break
        
        if i % print_progress_step == 0:
            print('Processed %d/%d rows, progress: %.3f%%' % (i, row_count, (i/row_count) * 100))

        if i is not None and i == limit:
            print('Early stop at i = %d' % i)
            break

    finish_time = time.time()
    print("\nExecuted in %s seconds" % (finish_time - start_time))

In [33]:
calc_n_keyword()

Processed 100000/1237973 rows, progress: 8.078%
Processed 200000/1237973 rows, progress: 16.155%
Processed 300000/1237973 rows, progress: 24.233%
Processed 400000/1237973 rows, progress: 32.311%
Processed 500000/1237973 rows, progress: 40.389%
Processed 600000/1237973 rows, progress: 48.466%
Processed 700000/1237973 rows, progress: 56.544%
Processed 800000/1237973 rows, progress: 64.622%
Processed 900000/1237973 rows, progress: 72.699%
Processed 1000000/1237973 rows, progress: 80.777%
Processed 1100000/1237973 rows, progress: 88.855%
Processed 1200000/1237973 rows, progress: 96.933%

Executed in 324.7123873233795 seconds


In [46]:
n_keyword

{'loop': 15621,
 'trolley': 1724,
 '2': 1123392,
 'mile': 2836,
 '3': 31779,
 '5': 15115,
 'km': 1093,
 '10': 2676,
 'station': 1785,
 'heritage': 403,
 'streetcar': 8,
 'line': 7954,
 'st': 33305,
 'louis': 0,
 'missouri': 2,
 'operated': 0,
 'november': 3,
 '2018': 0,
 'december': 1,
 '2019': 0,
 'bi': 127,
 'state': 0,
 'development': 1,
 'agency': 1,
 'regional': 2,
 'transit': 1,
 'operator': 0,
 'seeking': 0,
 'ways': 0,
 'restart': 0,
 'operations': 0,
 'opened': 0,
 'connected': 0,
 'delmar': 0,
 'district': 0,
 'history': 0,
 'museum': 0,
 'forest': 0,
 'park': 1,
 'serving': 1,
 'parts': 1,
 'university': 1,
 'city': 8,
 'neighborhoods': 0,
 'debaliviere': 0,
 'place': 12,
 'skinker': 0,
 'west': 0,
 'end': 14,
 'also': 14,
 'served': 2,
 'two': 16,
 'metrolink': 0,
 'light': 1,
 'rail': 2,
 'stations': 0,
 'closing': 0,
 'noon': 0,
 '6': 5,
 'p': 63,
 'thursdays': 0,
 'sundays': 0,
 'passengers': 0,
 'carried': 0,
 'replica': 0,
 'historic': 0,
 'streetcars': 0,
 'transporta

In [41]:
prob_keyword = n_keyword
for key, val in prob_keyword.items():
    prob_keyword[key] = 0

In [47]:
# calculate keyphrase probability for every word in prob_keyword
for key, val in n_word.items():
    
    if n_word[key] == 0:
        continue
        
    prob_keyword[key] = n_keyword[key] / n_word[key]

In [48]:
prob_keyword

{'loop': 1.0,
 'trolley': 1.0,
 '2': 1.0,
 'mile': 1.0,
 '3': 1.0,
 '5': 1.0,
 'km': 1.0,
 '10': 1.0,
 'station': 1.0,
 'heritage': 1.0,
 'streetcar': 1.0,
 'line': 1.0,
 'st': 1.0,
 'louis': 0,
 'missouri': 1.0,
 'operated': 0,
 'november': 1.0,
 '2018': 0,
 'december': 1.0,
 '2019': 0,
 'bi': 1.0,
 'state': 0,
 'development': 1.0,
 'agency': 1.0,
 'regional': 1.0,
 'transit': 1.0,
 'operator': 0,
 'seeking': 0,
 'ways': 0,
 'restart': 0,
 'operations': 0,
 'opened': 0,
 'connected': 0,
 'delmar': 0,
 'district': 0,
 'history': 0,
 'museum': 0,
 'forest': 0,
 'park': 1.0,
 'serving': 1.0,
 'parts': 1.0,
 'university': 1.0,
 'city': 1.0,
 'neighborhoods': 0,
 'debaliviere': 0,
 'place': 1.0,
 'skinker': 0,
 'west': 0,
 'end': 1.0,
 'also': 1.0,
 'served': 1.0,
 'two': 1.0,
 'metrolink': 0,
 'light': 1.0,
 'rail': 1.0,
 'stations': 0,
 'closing': 0,
 'noon': 0,
 '6': 1.0,
 'p': 1.0,
 'thursdays': 0,
 'sundays': 0,
 'passengers': 0,
 'carried': 0,
 'replica': 0,
 'historic': 0,
 'streetc